## Two Approaches to Yield Trading

For the start, I am focusing only on yield trading and not on Liquity provision strategies. The reason for that is yield trdaing is only price dependent which depends on the Implied APY (or market expectations) whwreas LP strategies also need to account for the relative value (reasonable, undervalued or over) of the Underlying APY in order to be successful. Once we can reasonable predict the direction of Implied APY we can then add on our models for the Underlying to create LP strategies. Let me explain that a bit further.

Looking at different markets on Pendle, I realised that they can be broadly divided into two categories:

1. Efficient Markets
2. Independent Markets

**Efficient markets** are those that I observe have the characteristic of closely following the Underlying APY with high liquidity and trading volume as a consequence. This should be expected of every market as we would presumne that the market reacting to the news and reward data would adjust in order to bring the Implied APY close to the Underlying APY so that large arbitrage is not possible. To look at an example we can look at gDAI:

![text](./gDAI.png)

The Underlying and Implied hug each other with a relatively low spread. Which indicates they may be [cointegrated](https://hudsonthames.org/an-introduction-to-cointegration/). We can create a simple strategy here such that when the underlying spreads out too much (let's say greater that $2\sigma$) we trade yt in the same direction of the momentum of the underlying.

Take another example of GLP:

![text](./GLP.png)

Various crossovers and follows the underlying with reasonable volatility. But here we see sharp jumps in the underlying which as I understand, GLP releases metrics on the platform fees that have been collected on a weekly basis and this jump indicates a new optimistic release but we don't see the market react as optimistically. Or at all during the next crash. So we need two things, even if the market is efficient:
1. An indicator for Underlying yield, mostly based on user activity, volume but opportunities are better when we can react first to collection news and other info that cause jumps.
2. We will have to combine that with and indicator of what the market on Pnedle is.

Now we come to the second type of markets on Pendle:

**Independent markets** are those that behave so unexpectedly or assymetrically to the Underlying APY that they can be assumed to be unaffected by it and controlled only by the market forces on Pendle. Let's take an example:

![text](./wstETH.png)

Looking at how the two diverge the Implied APY can in no way be explained by the Underlying and given the magnitude one can have a greater that 35% profit in a month, by longing YT. Of course this characteristic will be applicable to both markets with lower volume and liquidity but primarily to any market with a realtively large number of days to expiry as the gap between the Underlying and Implied has to close when expiry is near. This bring us to our next approach of identifying market forces on Pendle.

In [1]:
import requests
import numpy as np
import pandas as pd

Inspecting the Pendle API I see that there are two typoes of data available, one is tick data for the price of YT and PT and the APYs. But that doesn't give us any new information about the change in the market other than what the implied and volume is already telling us. Traditionally we would look at order flow but since we don't have a forward looking measure here we can suffice with the next best thing which are the past transactions. The best way that I can think of to deal with it is to keep it simple and create an [Order Flow Imbalance](https://quant.stackexchange.com/questions/43751/what-is-the-order-flow-imbalance) type indicator that tell us based on a range what the market is "flowing" towards.

![text](./OFI.png)

In [2]:
times = ''
list1 = []
api_url = f'https://api-v2.pendle.finance/core/v2/42161/transactions/?time_frame=day&timestamp_start=2024-02-18T00:00:00.000Z&limit=50&market=0xa0192f6567f8f5dc38c53323235fd08b318d2dca'
for i in range(1):
    response = requests.get(api_url)
    if response.status_code != 200:
        break
    response = response.json()
    data = response['results']
    #pd.DataFrame.from_dict(data)
    for dat in data:
        if dat['action'] == 'SWAP_PT' or dat['action'] == 'SWAP_YT':
            list1.append({'timestamp': dat['timestamp'], 'origin': dat['origin'], 'action': dat['action'], 'input': dat['inputs'][0]['asset']['name'], 
                          'input_amount': dat['inputs'][0]['amount'], 'input_price': dat['inputs'][0]['price']['usd'], 'output': dat['outputs'][0]['asset']['name'], 
                          'output_amount': dat['outputs'][0]['amount'], 'output_price': dat['outputs'][0]['price']['usd']})
    times = dat['timestamp']
    api_url = f'https://api-v2.pendle.finance/core/v2/42161/transactions/?time_frame=day&timestamp_end={times}&limit=50&market=0xa0192f6567f8f5dc38c53323235fd08b318d2dca'
df = pd.DataFrame.from_dict(list1)
df['input_value'] = df['input_amount'] * df['input_price']
df['output_value'] = df['output_amount'] * df['output_price']
df

,timestamp,origin,action,input,input_amount,input_price,output,output_amount,output_price,input_value,output_value
0,2024-02-19T00:48:09.000Z,ROUTER,SWAP_PT,PT gDAI 28MAR2024,1018.500000,0.982036,Gains Network DAI,875.420869,1.142421,1000.203819,1000.099622
1,2024-02-18T20:24:56.000Z,ROUTER,SWAP_YT,Dai Stablecoin,100.000000,0.999920,YT gDAI 28MAR2024,5508.086813,0.018050,99.992002,99.419648
2,2024-02-18T20:09:35.000Z,ROUTER,SWAP_YT,Tether USD,10001.174719,1.000210,YT gDAI 28MAR2024,558355.978352,0.018044,10003.274966,10074.806208
3,2024-02-18T19:11:24.000Z,ROUTER,SWAP_PT,USD Coin,500.000000,1.000062,PT gDAI 28MAR2024,508.730474,0.982638,500.030915,499.897778
4,2024-02-18T15:09:53.000Z,ROUTER,SWAP_YT,SY gDAI,21.781787,1.142304,YT gDAI 28MAR2024,1424.425219,0.017363,24.881416,24.732491
5,2024-02-18T15:02:58.000Z,ROUTER,SWAP_PT,Dai Stablecoin,200.016364,1.000026,PT gDAI 28MAR2024,203.522382,0.982663,200.021514,199.993987
6,2024-02-18T14:52:18.000Z,ROUTER,SWAP_YT,SY gDAI,65.015650,1.142598,YT gDAI 28MAR2024,4251.017049,0.017371,74.286738,73.842977
7,2024-02-18T14:16:44.000Z,ROUTER,SWAP_PT,Dai Stablecoin,9998.900000,1.000287,PT gDAI 28MAR2024,10174.094026,0.982914,10001.766485,10000.255378
8,2024-02-18T13:16:13.000Z,ROUTER,SWAP_YT,USD Coin,1004.774101,1.000062,YT gDAI 28MAR2024,57376.305277,0.017405,1004.836226,998.640201
9,2024-02-18T12:30:08.000Z,ROUTER,SWAP_PT,PT gDAI 28MAR2024,1.638382,0.982786,ETH,0.000572,2809.268505,1.610180,1.607599


A naive approach would be to simply take the difference of the value of each YT and PT.

In [3]:
v_yt = np.sum(df[df['output'] == 'YT gDAI 28MAR2024']['output_value']) - np.sum(df[df['input'] == 'YT gDAI 28MAR2024']['input_value'])
v_pt = np.sum(df[df['output'] == 'PT gDAI 28MAR2024']['output_value']) - np.sum(df[df['input'] == 'PT gDAI 28MAR2024']['input_value'])
oi = v_pt - v_yt
oir = oi / (v_pt + v_yt)
print(f"Order Imbalance is {oi} on PT side")
print(f"Order Imbalance Ratio is {oir}")

Order Imbalance is -10223.54563526644 on PT side
Order Imbalance Ratio is -0.3178835316341849


We can normalise this value by a rolling window of let's say 7 days to get better reuslts in scale. But there's more to account for like the volume of liquidity being added and being removed which would indicate both trust in the Pendle protocol and the sentiment of users. I haven't figured out how to incorporate this part into the equation and what weightage to give it. 

In [4]:
times = ''
list1 = []
api_url = f'https://api-v2.pendle.finance/core/v2/42161/transactions/?time_frame=day&timestamp_start=2024-02-18T00:00:00.000Z&limit=50&market=0xa0192f6567f8f5dc38c53323235fd08b318d2dca'
for i in range(1):
    response = requests.get(api_url)
    if response.status_code != 200:
        break
    response = response.json()
    data = response['results']
    #pd.DataFrame.from_dict(data)
    for dat in data:
        if dat['action'] == 'ADD_LIQUIDITY' or dat['action'] == 'REMOVE_LIQUIDITY':
            list1.append({'timestamp': dat['timestamp'], 'origin': dat['origin'], 'action': dat['action'], 'input': dat['inputs'][0]['asset']['name'], 
                          'input_amount': dat['inputs'][0]['amount'], 'input_price': dat['inputs'][0]['price']['usd'], 'output': dat['outputs'][0]['asset']['name'], 
                          'output_amount': dat['outputs'][0]['amount'], 'output_price': dat['outputs'][0]['price']['usd']})
    times = dat['timestamp']
    api_url = f'https://api-v2.pendle.finance/core/v2/42161/transactions/?time_frame=day&timestamp_end={times}&limit=50&market=0xa0192f6567f8f5dc38c53323235fd08b318d2dca'
df = pd.DataFrame.from_dict(list1)
df['input_value'] = df['input_amount'] * df['input_price']
df['output_value'] = df['output_amount'] * df['output_price']
df

,timestamp,origin,action,input,input_amount,input_price,output,output_amount,output_price,input_value,output_value
0,2024-02-18T21:28:07.000Z,ROUTER,ADD_LIQUIDITY,Dai Stablecoin,139.445968,0.999952,Pendle Market,64.081319,2.175543,139.439283,139.411664
1,2024-02-18T20:23:32.000Z,ROUTER,ADD_LIQUIDITY,Dai Stablecoin,1000.000000,0.999920,YT gDAI 28MAR2024,487.051732,0.018039,999.920020,8.786003
2,2024-02-18T16:07:10.000Z,ROUTER,ADD_LIQUIDITY,Dai Stablecoin,130.935911,1.000038,Pendle Market,60.146585,2.176226,130.940857,130.892550
3,2024-02-18T16:05:46.000Z,ROUTER,ADD_LIQUIDITY,Dai Stablecoin,3045.979754,1.000038,Pendle Market,1402.893852,2.176225,3046.094800,3053.012759
4,2024-02-18T15:10:51.000Z,ROUTER,ADD_LIQUIDITY,PT gDAI 28MAR2024,203.522382,0.982663,Pendle Market,91.894970,2.176176,199.993851,199.979644
5,2024-02-18T14:13:48.000Z,ROUTER,REMOVE_LIQUIDITY,Pendle Market,944.353906,2.176704,Gains Network DAI,1798.965489,1.142590,2055.579000,2055.479734
6,2024-02-18T13:40:49.000Z,ROUTER,REMOVE_LIQUIDITY,Pendle Market,1.999302,2.175993,USD Coin (Arb1),4.341626,0.999970,4.350466,4.341496
7,2024-02-18T12:33:04.000Z,ROUTER,ADD_LIQUIDITY,Gains Network DAI,1750.844465,1.142362,Pendle Market,918.947333,2.176364,2000.097578,1999.964163
8,2024-02-18T11:38:29.000Z,ROUTER,ADD_LIQUIDITY,Dai Stablecoin,10000.000000,1.000230,Pendle Market,4594.942128,2.176641,10002.304300,10001.538346
9,2024-02-18T10:23:36.000Z,ROUTER,ADD_LIQUIDITY,Dai Stablecoin,32.297635,1.000640,Pendle Market,14.838615,2.177490,32.318314,32.310929


The goal is to combine both of these into an oder imbalance sort of indicator than can work as a proxy for order flow. Below minting and redeeming volumes are small enough to be conveniently ignored.

In [5]:
times = ''
list1 = []
api_url = f'https://api-v2.pendle.finance/core/v2/42161/transactions/?time_frame=day&limit=50&market=0xa0192f6567f8f5dc38c53323235fd08b318d2dca'
for i in range(7):
    response = requests.get(api_url)
    if response.status_code != 200:
        break
    response = response.json()
    data = response['results']
    #pd.DataFrame.from_dict(data)
    for dat in data:
        if dat['action'] == 'MINT_PY' or dat['action'] == 'REDEEM_PY':
            list1.append({'timestamp': dat['timestamp'], 'origin': dat['origin'], 'action': dat['action'], 'input': dat['inputs'][0]['asset']['name'], 
                          'input_amount': dat['inputs'][0]['amount'], 'input_price': dat['inputs'][0]['price']['usd'], 'output': dat['outputs'][0]['asset']['name'], 
                          'output_amount': dat['outputs'][0]['amount'], 'output_price': dat['outputs'][0]['price']['usd']})
    times = dat['timestamp']
    api_url = f'https://api-v2.pendle.finance/core/v2/42161/transactions/?time_frame=day&timestamp_end={times}&limit=50&market=0xa0192f6567f8f5dc38c53323235fd08b318d2dca'
df = pd.DataFrame.from_dict(list1)
df['input_value'] = df['input_amount'] * df['input_price']
df['output_value'] = df['output_amount'] * df['output_price']
df

,timestamp,origin,action,input,input_amount,input_price,output,output_amount,output_price,input_value,output_value
0,2024-02-17T04:20:21.000Z,ROUTER,REDEEM_PY,PT gDAI 28MAR2024,10047.273948,0.982533,Gains Network DAI,8800.384279,1.141572,9871.773652,10046.269221
1,2024-02-16T14:27:02.000Z,ROUTER,REDEEM_PY,PT gDAI 28MAR2024,98665.576484,0.982260,Gains Network DAI,86453.785987,1.141486,96915.276164,98685.768903
2,2024-02-16T14:23:53.000Z,ROUTER,REDEEM_PY,PT gDAI 28MAR2024,98.665576,0.982390,Gains Network DAI,86.453810,1.141486,96.928060,98.685796
3,2024-02-15T10:20:48.000Z,ROUTER,REDEEM_PY,PT gDAI 28MAR2024,61.278976,0.982709,ETH,0.021835,2790.404359,60.219431,60.927281
4,2024-02-15T10:18:33.000Z,ROUTER,MINT_PY,SY gDAI,53.724116,1.140737,PT gDAI 28MAR2024,61.278976,0.982709,61.285099,60.219391
5,2024-02-15T02:35:03.000Z,ROUTER,REDEEM_PY,PT gDAI 28MAR2024,10514.856296,0.983639,Gains Network DAI,9219.245523,1.140692,10342.818051,10516.316585
6,2024-02-14T18:27:01.000Z,ROUTER,MINT_PY,Gains Network DAI,9.098413,1.140162,PT gDAI 28MAR2024,10.374143,0.982842,10.373666,10.196148
7,2024-02-11T21:18:18.000Z,ROUTER,REDEEM_PY,PT gDAI 28MAR2024,279.578010,0.983195,USD Coin,279.488234,1.000024,274.879756,279.495054
8,2024-02-11T11:51:16.000Z,ROUTER,REDEEM_PY,PT gDAI 28MAR2024,285.880724,0.983008,Gains Network DAI,251.217364,1.138086,281.023076,285.906907
9,2024-02-11T11:50:06.000Z,ROUTER,MINT_PY,SY gDAI,251.217364,1.138086,PT gDAI 28MAR2024,285.880724,0.983008,285.906951,281.022990


Once we have an indicator for the Pendle Market which should be generalizable we can then take the model of the Underlying, which may be specific, and combine it to create an LP strategy out of it. This is what I have been working on please feel free to add in your comments or questions.